In [ ]:
# Mount to drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
path = "/Apple-Invariant-feature/VisualizationExample/"
os.chdir(path)
os.getcwd()

##segmentation process

In [ ]:
from PIL import Image

rottenimg_source = path + 'rotten_001_source.jpg'
jpg_image = Image.open(rottenimg_source)

# Save the image in PNG format
png_image_path = "rotten_001_source.png"
jpg_image.save(png_image_path, format="PNG")

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

rottenimg_source = path + 'rotten_001_source.png'
rottenimg_graylevel = path + 'rotten_001.png'
rotten_mask = path + 'mask_rotten_001.png'
image1_source = cv2.imread(rottenimg_source)
image1_graylevel = cv2.imread(rottenimg_graylevel)
image1_mask = cv2.imread(rotten_mask)
image1_out = np.zeros_like(image1_graylevel)
image1_out[image1_mask > 0] = image1_graylevel[image1_mask > 0]
cv2_imshow(image1_source)
cv2_imshow(image1_graylevel)
cv2_imshow(image1_mask)
cv2_imshow(image1_out)

In [ ]:
cv2_imshow(image1_source)
cv2_imshow(image1_graylevel)
cv2_imshow(image1_mask)
cv2_imshow(image1_out)

##visulazation map

In [ ]:
# image and mask visualization
import cv2
freshimg_path = path + '1_021.png'
freshmaskimg_path = path + 'mask_1_021.png'
image1 = cv2.imread(freshimg_path, cv2.IMREAD_GRAYSCALE)
image1_mask = cv2.imread(freshmaskimg_path, cv2.IMREAD_GRAYSCALE)

In [ ]:
import numpy as np
from google.colab.patches import cv2_imshow

image1_out = np.zeros_like(image1)
image1_out[image1_mask > 0] = image1[image1_mask > 0]

cv2_imshow(image1_out)

In [ ]:
freshimg_path = path + 'rotten_001.png'
freshmaskimg_path = path + 'mask_rotten_001.png'
image2 = cv2.imread(freshimg_path, cv2.IMREAD_GRAYSCALE)
image2_mask = cv2.imread(freshmaskimg_path, cv2.IMREAD_GRAYSCALE)

image2_out = np.zeros_like(image2)
image2_out[image2_mask > 0] = image2[image2_mask > 0]

cv2_imshow(image2_out)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
cax = plt.imshow(image1, cmap='gray', vmin=0, vmax=250)
colorbar = plt.colorbar(cax, fraction=0.048, pad=0.04)
colorbar.ax.tick_params(labelsize=14)
# Make the color bar labels bold
for label in colorbar.ax.get_yticklabels():
    label.set_weight('bold')

plt.axis('off')
plt.show()

# Display the rescaled image (image2) with a smaller color bar
plt.figure(figsize=(6, 6))
cax = plt.imshow(image1, cmap='gray', vmin=0, vmax=200)
colorbar = plt.colorbar(cax, fraction=0.048, pad=0.04)
colorbar.ax.tick_params(labelsize=14)
# Make the color bar labels bold
for label in colorbar.ax.get_yticklabels():
    label.set_weight('bold')

plt.axis('off')
plt.show()

In [ ]:
!pip install pyradiomics

In [ ]:
from skimage import io

img1_path = path + 'image1.png'
Image1 = io.imread(img1_path)

num_pixel1 = Image1.shape[0]
num_pixel2 = Image1.shape[1]
print(f"The image has {num_pixel1} pixels.")
print(f"The image has {num_pixel2} pixels.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import generic_filter
from skimage import io, color, util
from skimage.feature import graycomatrix, graycoprops
import math

def glcm_contrast(region):
    # Calculate the contrast
    glcm = graycomatrix(region, [1], [0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    return contrast

Image1 = io.imread(img1_path)
# Check if the image has an alpha channel (fourth channel)
if Image1.shape[-1] == 4:
    # Convert RGBA to RGB by removing the alpha channel
    Image1 = Image1[:, :, :3]

Image1_gray = color.rgb2gray(Image1)  # Convert to grayscale
Image1_gray_uint8 = util.img_as_ubyte(Image1_gray)  # Convert to unsigned 8-bit integer

window_size = 15
image_height, image_width = Image1_gray_uint8.shape
result1_invariant = np.zeros_like(Image1_gray_uint8, dtype=float)
result1_original = np.zeros_like(Image1_gray_uint8, dtype=float)

# Iterate through the image moving the window pixel by pixel-invariant
for i in range(image_height - window_size + 1):
    for j in range(image_width - window_size + 1):
        window = Image1_gray_uint8[i:i+window_size, j:j+window_size]
        result1_invariant[i + window_size // 2, j + window_size // 2] = glcm_contrast(window)/math.log(200)

# Iterate through the image moving the window pixel by pixel-original
for i in range(image_height - window_size + 1):
    for j in range(image_width - window_size + 1):
        window = Image1_gray_uint8[i:i+window_size, j:j+window_size]
        result1_original[i + window_size // 2, j + window_size // 2] = glcm_contrast(window)

In [ ]:
max_value = np.max(result1_invariant)
min_value = np.min(result1_invariant)

print("Max Value:", max_value)
print("Min Value:", min_value)

In [ ]:
max_value = np.max(result1_original)
min_value = np.min(result1_original)

print("Max Value:", max_value)
print("Min Value:", min_value)

In [ ]:
plt.figure(figsize=(8, 8))

cax = plt.imshow(result1_original, cmap='gray', vmin=0, vmax=80)

colorbar = plt.colorbar(cax, fraction=0.048, pad=0.04)
colorbar.ax.tick_params(labelsize=14)
# Make the color bar labels bold
for label in colorbar.ax.get_yticklabels():
    label.set_weight('bold')

plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

cax = plt.imshow(result1_invariant, cmap='gray', vmin=0, vmax=15)

colorbar_ticks = [0, 5, 10, 15, 20]

colorbar = plt.colorbar(cax, fraction=0.048, pad=0.04, ticks=colorbar_ticks)
colorbar.ax.tick_params(labelsize=14)
# Make the color bar labels bold
for label in colorbar.ax.get_yticklabels():
    label.set_weight('bold')

plt.axis('off')

plt.show()

In [ ]:
from skimage import io

img2_path = path + 'image2.png'
Image2 = io.imread(img2_path)

num_pixel1 = Image2.shape[0]
num_pixel2 = Image2.shape[1]
print(f"The image has {num_pixel1} pixels.")
print(f"The image has {num_pixel2} pixels.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import generic_filter
from skimage import io, color, util
from skimage.feature import graycomatrix, graycoprops
import math

def glcm_contrast(region):
    # Calculate the contrast
    glcm = graycomatrix(region, [1], [0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    return contrast

Image2 = io.imread(img2_path)
# Check if the image has an alpha channel (fourth channel)
if Image2.shape[-1] == 4:
    # Convert RGBA to RGB by removing the alpha channel
    Image2 = Image2[:, :, :3]

Image2_gray = color.rgb2gray(Image2)  # Convert to grayscale
Image2_gray_uint8 = util.img_as_ubyte(Image2_gray)  # Convert to unsigned 8-bit integer

window_size = 15
image_height, image_width = Image2_gray_uint8.shape
result2_invariant = np.zeros_like(Image2_gray_uint8, dtype=float)
result2_original = np.zeros_like(Image2_gray_uint8, dtype=float)

# Iterate through the image moving the window pixel by pixel - invariant
for i in range(image_height - window_size + 1):
    for j in range(image_width - window_size + 1):
        window = Image2_gray_uint8[i:i+window_size, j:j+window_size]
        result2_invariant[i + window_size // 2, j + window_size // 2] = glcm_contrast(window)/math.log(10)

# Iterate through the image moving the window pixel by pixel - original
for i in range(image_height - window_size + 1):
    for j in range(image_width - window_size + 1):
        window = Image2_gray_uint8[i:i+window_size, j:j+window_size]
        result2_original[i + window_size // 2, j + window_size // 2] = glcm_contrast(window)

In [ ]:
max_value = np.max(result2_original)
min_value = np.min(result2_original)

print("Max Value:", max_value)
print("Min Value:", min_value)

In [ ]:
max_value = np.max(result2_invariant)
min_value = np.min(result2_invariant)

print("Max Value:", max_value)
print("Min Value:", min_value)

In [ ]:
# Show the image
plt.figure(figsize=(8, 8))

cax = plt.imshow(result2_original, cmap='gray', vmin=0, vmax=150)

plt.colorbar(cax, fraction=0.048, pad=0.04)
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

cax = plt.imshow(result2_invariant, cmap='gray', vmin=0, vmax=90)

plt.colorbar(cax, fraction=0.048, pad=0.04)
plt.axis('off')
plt.show()

In [ ]:
def glcm_dissimilarity(region):
    # Calculate the dissimilarity - Difference Average
    glcm = graycomatrix(region, [1], [0], symmetric=True, normed=True)
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    return dissimilarity

Image1 = io.imread(img1_path)
# Check if the image has an alpha channel (fourth channel)
if Image1.shape[-1] == 4:
    # Convert RGBA to RGB by removing the alpha channel
    Image1 = Image1[:, :, :3]

Image1_gray = color.rgb2gray(Image1)  # Convert to grayscale
Image1_gray_uint8 = util.img_as_ubyte(Image1_gray)  # Convert to unsigned 8-bit integer

window_size = 15
image_height, image_width = Image1_gray_uint8.shape
result3_original = np.zeros_like(Image1_gray_uint8, dtype=float)
result3_invariant = np.zeros_like(Image1_gray_uint8, dtype=float)

# Iterate through the image moving the window pixel by pixel - original
for i in range(image_height - window_size + 1):
    for j in range(image_width - window_size + 1):
        window = Image1_gray_uint8[i:i+window_size, j:j+window_size]
        result3_original[i + window_size // 2, j + window_size // 2] = glcm_dissimilarity(window)

# Iterate through the image moving the window pixel by pixel - invariant
for i in range(image_height - window_size + 1):
    for j in range(image_width - window_size + 1):
        window = Image1_gray_uint8[i:i+window_size, j:j+window_size]
        result3_invariant[i + window_size // 2, j + window_size // 2] = glcm_dissimilarity(window) /math.log(200)

In [ ]:
max_value = np.max(result3_original)
min_value = np.min(result3_original)

print("Max Value:", max_value)
print("Min Value:", min_value)

In [ ]:
max_value = np.max(result3_invariant)
min_value = np.min(result3_invariant)

print("Max Value:", max_value)
print("Min Value:", min_value)

In [ ]:
plt.figure(figsize=(8, 8))

cax = plt.imshow(result3_original, cmap='gray', vmin=0, vmax=8)

colorbar = plt.colorbar(cax, fraction=0.048, pad=0.04)
colorbar.ax.tick_params(labelsize=14)
# Make the color bar labels bold
for label in colorbar.ax.get_yticklabels():
    label.set_weight('bold')

plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

cax = plt.imshow(result3_invariant, cmap='gray', vmin=0, vmax=2)

colorbar = plt.colorbar(cax, fraction=0.048, pad=0.04)
colorbar.ax.tick_params(labelsize=14)
# Make the color bar labels bold
for label in colorbar.ax.get_yticklabels():
    label.set_weight('bold')

plt.axis('off')
plt.show()

In [ ]:
from skimage import io

img2_path = path + 'image2.png'
Image2 = io.imread(img2_path)

num_pixel1 = Image2.shape[0]
num_pixel2 = Image2.shape[1]
print(f"The image has {num_pixel1} pixels.")
print(f"The image has {num_pixel2} pixels.")

In [ ]:
def glcm_dissimilarity(region):
    # Calculate the dissimilarity - Difference Average
    glcm = graycomatrix(region, [1], [0], symmetric=True, normed=True)
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    return dissimilarity

Image1 = io.imread(img1_path)
# Check if the image has an alpha channel (fourth channel)
if Image1.shape[-1] == 4:
    # Convert RGBA to RGB by removing the alpha channel
    Image1 = Image1[:, :, :3]

Image1_gray = color.rgb2gray(Image1)  # Convert to grayscale
Image1_gray_uint8 = util.img_as_ubyte(Image1_gray)  # Convert to unsigned 8-bit integer

window_size = 15
image_height, image_width = Image1_gray_uint8.shape
result4_original = np.zeros_like(Image1_gray_uint8, dtype=float)
result4_invariant = np.zeros_like(Image1_gray_uint8, dtype=float)

# Iterate through the image moving the window pixel by pixel - original
for i in range(image_height - window_size + 1):
    for j in range(image_width - window_size + 1):
        window = Image1_gray_uint8[i:i+window_size, j:j+window_size]
        result4_original[i + window_size // 2, j + window_size // 2] = glcm_dissimilarity(window)

# Iterate through the image moving the window pixel by pixel - invariant
for i in range(image_height - window_size + 1):
    for j in range(image_width - window_size + 1):
        window = Image1_gray_uint8[i:i+window_size, j:j+window_size]
        result4_invariant[i + window_size // 2, j + window_size // 2] = glcm_dissimilarity(window) /math.log(10)

In [ ]:
max_value = np.max(result4_original)
min_value = np.min(result4_original)

print("Max Value:", max_value)
print("Min Value:", min_value)

In [ ]:
max_value = np.max(result4_invariant)
min_value = np.min(result4_invariant)

print("Max Value:", max_value)
print("Min Value:", min_value)

In [ ]:
plt.figure(figsize=(8, 8))

cax = plt.imshow(result4_original, cmap='gray', vmin=0, vmax=10)

colorbar = plt.colorbar(cax, fraction=0.048, pad=0.04)
colorbar.ax.tick_params(labelsize=14)
# Make the color bar labels bold
for label in colorbar.ax.get_yticklabels():
    label.set_weight('bold')

plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

cax = plt.imshow(result4_invariant, cmap='gray', vmin=0, vmax=5)

colorbar = plt.colorbar(cax, fraction=0.048, pad=0.04)
colorbar.ax.tick_params(labelsize=14)
# Make the color bar labels bold
for label in colorbar.ax.get_yticklabels():
    label.set_weight('bold')

plt.axis('off')
plt.show()